In [ ]:
%pip install langchain openai chromadb langchain-community -q

In [1]:
import chromadb

In [2]:
client = chromadb.HttpClient(
    host = "3.35.104.197",
    port = 10090,
)

In [3]:
client.heartbeat()

1741656260917843120

In [4]:
client.list_collections()


['multilingual-e5-large', 'test', 'hyqe', 'data', 'bge-m3', 'benchmark_data']

In [84]:
collection = client.get_collection("multilingual-e5-large")

In [ ]:
# # 메타데이터 기반 검색
result1 = collection.get(
    where={"doc_id": 'f3017cfc-9e94-49bb-af25-4303d6718ca5'}
)
pprint(result1)

## Tagging chain

In [ ]:
from langchain_chroma import Chroma
from dotenv import load_dotenv
load_dotenv()

COLLECTION_NAME = "collection"
PERSIST_DIRECTORY = "vector_store/collection1"


In [ ]:
from langchain_openai import ChatOpenAI
from langchain.chains import create_tagging_chain
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o", temperature=0)

# 고객요청의도 추출을 위한 schema 설정
schema = {
    "properties": {
        "scope": {
            "type": "string",
            "enum": ['EXTERNAL',
                    'INTERNAL',
            ]
        }
    }
}
extraction_chain = create_tagging_chain(schema, llm)

In [ ]:
# embedding
from langchain_openai import OpenAIEmbeddings
embedding_model = OpenAIEmbeddings(model="text-embedding-3-small")

# vector store
vector_store1 = Chroma.from_documents(
    documents=docs1,
    embedding=embedding_model,
    collection_name=COLLECTION_NAME+'1',
    persist_directory=PERSIST_DIRECTORY
)
vector_store2 = Chroma.from_documents(
    documents=docs2, 
    embedding=embedding_model, 
    collection_name=COLLECTION_NAME+'2',
    persist_directory=PERSIST_DIRECTORY
)
vector_store3 = Chroma.from_documents(
    documents=docs3, 
    embedding=embedding_model, 
    collection_name=COLLECTION_NAME+'3',
    persist_directory=PERSIST_DIRECTORY
)

In [ ]:
retriever1 = vector_store1.as_retriever(search_kwargs={"k": 1})
retriever2 = vector_store2.as_retriever(search_kwargs={"k": 1})
retriever3 = vector_store3.as_retriever(search_kwargs={"k": 1})

In [ ]:
retrievers = {
    "": retriever1,
    "EXTERNAL": retriever2,
    "INTERNAL": retriever3,
}

In [ ]:
from langchain_core.runnables import chain

@chain
def custom_chain(question):
    response = extraction_chain.invoke(question)
    #print(response.get('scope', ""))
    retriever = retrievers[response.get('scope') or ""]
    return retriever.invoke(question)

In [ ]:
from langchain.prompts import PromptTemplate

template = """다음과 같은 맥락과 히스토리를 사용하여 질문에 대답하십시오.
맥락: {context}
질문: {question}
도움이 되는 답변:"""

rag_prompt_custom = PromptTemplate.from_template(template)

In [ ]:
def using_intent_get_relevant_document(user_question):
    relevant_document = custom_chain.invoke(user_question)[0]
    return relevant_document

In [ ]:
# RAG chain 설정
from langchain.schema.runnable import RunnablePassthrough

rag_chain = {"context": using_intent_get_relevant_document, "question": RunnablePassthrough()} | rag_prompt_custom | llm

## OpenAI 조회

In [8]:
from langchain_openai import ChatOpenAI
from dotenv import load_dotenv
load_dotenv()

llm = ChatOpenAI(model_name="gpt-4o", temperature=0, model_kwargs={"stream": True})

In [2]:
from langchain_openai.embeddings import OpenAIEmbeddings
import chromadb
from langchain_chroma import Chroma
from transformers import AutoTokenizer

In [63]:
embedding_function = OpenAIEmbeddings(
    base_url = "https://76mqtyy2wie64y-10050.proxy.runpod.net/v1",
    model = "multilingual-e5-large",
    api_key="team3",
    tiktoken_enabled=False,
    embedding_ctx_length = 502
)

client = chromadb.HttpClient(
    host="3.35.104.197",
    port=10090
    )

vector_store = Chroma(
    collection_name="multilingual-e5-large", 
    embedding_function=embedding_function, 
    client=client
)

In [68]:
retriever = vector_store.as_retriever(search_kwargs={"k":1})

In [73]:
from langchain.chains import RetrievalQA
from langchain.prompts import PromptTemplate
from langchain_chroma import Chroma
from pprint import pprint

prompt_template = PromptTemplate(
    input_variables=["context", "question"],
    template="""
    직원들에 질문에 아래 문맥을 참고하여 응답을 해주었으면 좋겠어요.
    문맥에는 회사에 대한 내용입니다. 모르는 내용일 경우 모른다고 정확하게 명시해 주세요. 
    
    문맥:\n{context}
    
    
    질문: {question}\n\n답변:
    """
)

def rag_chain(question: str):
    llm = ChatOpenAI(model="gpt-4-turbo",)
        
    docs = retriever.get_relevant_documents(question) 
    context = "\n".join([doc.page_content for doc in docs])
    pprint(docs)
        
    qa_chain = RetrievalQA.from_chain_type(
        llm=llm,
        retriever=retriever,
        chain_type="stuff",
        chain_type_kwargs={"prompt": prompt_template}
    )
    
    return qa_chain.invoke(question)


In [ ]:
result = rag_chain('주름개선 제품 소개해줘')

내가 불러왔는데 한 문서에 어마어마하게 들어가있는건가 싶어서

[Document(metadata={'doc_id': 'f3017cfc-9e94-49bb-af25-4303d6718ca5', 'source': "SHIELD FOR URBAN POLLUTION 꼼꼼히 따져본 쿠션 팩트 리뷰 리뉴얼의 축복이 끝이 없네! 레전드 뷰티템의 최신 근황 묵직한 클래식 오일 vs 워터 또는 세럼과 결합한 오일-IN 뷰티 전문가들의 솔직한 쿠션 리뷰 어버이날 선물 얼루어 추천! 올리브영 베스트 아이템 1 컬러 선택만 잘해도 인생 파데 타고난 피부의 운명 피부 철벽 방어를 위한 '멀티형 자외선 차단제' 피부도 숙취 해소 환절기 피부를 좌우하는 나이트 케어 제품 4 황사와 미세먼지에 맞설 효소 파우더 클렌저"}, page_content="BEAUTY\nSHIELD FOR URBAN POLLUTION\n2019.03.04 by 박정인\n복사 공유\n각종 유해 물질이 가득한 도심 속에서 내 피부를 현명하게 지키는 방법.\nMISSION 1\n유해 환경으로부터 피부 철벽 차단\n하루 종일 도심에서 일하는 현대인은 공기 중의 미세먼지와 황사, 중금속에 쉽게 노출된다. 해마다 점점 더 심\n해지는 다양한 유해 요소로부터 피부를 보호해줄 방어막이 필요한데, 최근 몇 년 사이 이러한 역할을 하는 아이\n템이 대거 출시되는 추세다. 유해 물질과 피부 접촉을 차단하는 아이템을 사용하기만 해도 피부에 가해지는 자\n극을 훨씬 줄일 수 있고, 나아가 피부 노화까지 예방할 수 있다.\nhttps://www.allurekorea.com/2019/03/04/shield-for-urban-pollution/ 1/12\n25. 2. 7. 오후 5:39 SHIELD FOR URBAN POLLUTION | 얼루어 코리아 (Allure Korea)\n블랙 터틀넥은 코스(Cos), 골드 이어링은 1064 스튜디오(1064 Studios), 링은 제이 그레이슬렛(J.Gracelet).\n광고\n1 랑콤의 UV 엑스퍼트 톤업 밀크 로지 블룸 자외선은 기본, 초미세먼지까지 차단하는 

In [71]:
result

{'query': '미백제품  소개해줘',
 'result': '미백 제품에 대해 관심이 많으신 것 같네요. 미백 제품은 일반적으로 피부의 색소 침착을 감소시키고 피부 톤을 밝게 하는 데 도움을 줍니다. 다양한 브랜드에서 여러 종류의 미백 제품을 제공하고 있으며, 이 중에서도 특히 인기 있는 몇 가지 제품을 소개해드리겠습니다.\n\n1. **라네즈의 화이트 듀 스킨 리파이너**\n   - 라네즈의 화이트 듀 시리즈는 피부 톤을 밝게 해주고, 보습을 도와주면서 피부를 맑고 투명하게 가꿔줍니다.\n\n2. **아모레퍼시픽의 타임레스폰스 스킨 리뉴얼 세럼**\n   - 고급스러운 이 세럼은 피부 노화의 징후를 개선하고, 피부 톤을 균일하게 해주며 미백 효과를 제공합니다.\n\n3. **숨37°의 시크릿 에센스**\n   - 이 에센스는 피부의 자연스러운 광채를 불어넣어주고, 피부 톤을 밝게 해주며 미백 효과가 있습니다.\n\n4. **에스티 로더의 사이버 화이트 브릴리언트 셀**\n   - 높은 미백 효과를 지닌 이 제품은 피부의 어두운 반점을 명확히 개선해주고, 전반적인 피부 톤을 밝게 해줍니다.\n\n미백 제품을 선택할 때는 자신의 피부 타입과 필요에 맞는 제품을 선택하는 것이 중요합니다. 또한, 제품에 포함된 성분을 확인하여 피부에 자극을 줄 수 있는 성분이 없는지도 확인해야 합니다. 사용하기 전에 소량을 테스트하여 피부 반응을 확인하는 것도 좋은 방법입니다.'}